In [1]:
import os
import sys
import pandas as pd

import evo_helpers as hlp
import evo_constants as CTS

In [80]:
hlp.welcome()

Welcome to EvoMinig

Press enter to continune ...


In [ ]:
#from importlib import reload
#reload(evo_helpers)

opt = hlp.option_menu()

Press 1 to continue with precharged example.
Press 2 to continue with your own databases.


In [ ]:
class Evomining:
    """This class creates an evomining object.
    Its methods permit to access to the evomining
    funtionality.
    """

    def __init__(self, centralDB,
                evo_genomesDB,
                nat_prodsDB):
        self.central = centralDB
        self.genomesDB = evo_genomesDB
        self.nat_prods = nat_prodsDB

In [ ]:
## obtener las bases de datos del usuario

#from importlib import reload
#reload(evo_helpers)

if opt == 1:
    hlp.make_all_evo_headers(CTS.EXMPL_DB["genomes"],
                                 CTS.EXMPL_DB["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    evo_genomes_path = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(cts.EXML_DB["central"],
                          cts.EXMPL_DB["genomes"],
                          cts.EXMPL_DB["nat_prods"])
    #evomining.run()

else: 
    user_dbs = hlp.get_dbs()
    hlp.make_all_evo_headers(user_dbs["genomes"], 
                             user_dbs["rast_ids"])
    hlp.join_evo_headers_files()
    hlp.remove_evo_headers_files()
    
    user_dbs["genomes"] = CTS.EVO_GENOMES_DB
    
    evomining = EvoMining(user_dbs["central"],
                         user_dbs["genomes"],
                         user_dbs["nat_prods"])
    #evomining.run()

#evomining.run()

## Obtener los archivos .faa con los encabezados en formato evomining:
### Los nuevos .faa individuales se guardan en data/aux_data/  (se borran una vez utilizados)
### La base de datos con encabezados de evomining se guarda en /data/data_bases/evo_genomes_db/

In [ ]:
#from importlib import reload
#reload(evo_helpers)

genomes_db = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17"
Rast_ids = "/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/example/genomes/los17Rast.ids"
hlp.make_all_evo_headers(genomes_db, Rast_ids)
hlp.join_evo_headers_files()
hlp.remove_evo_headers_files()

## Aplicar makeblastdb a las bases de datos que necesitaremos:

In [ ]:
hlp.makeblast_db(CTS.EVO_GENOMES_DB)

In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["central"])


In [ ]:
hlp.makeblast_db(CTS.EXMPL_DB["nat_prods"])

## Obtener los blastp entre la base de datos central vs la base de datos de genomas:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'evo_genomes_db_blastdb/'
            
hlp.apply_blastp(CTS.EXMPL_DB["central"], blastdb_path)

In [ ]:
CTS.EVO_GENOMES_DB

## Obtener los blastp entre la base de datos de genomas vs la base de datos central:

In [ ]:
blastdb_path = CTS.BLASTDBs_PATH + 'central_blastdb/'
            
hlp.apply_blastp(CTS.EVO_GENOMES_DB, blastdb_path)

## Obtener los BBH:

### Obtener los mejores elementos únicos del blast de metabolismo central vs genomas: 
### Estos se guardarán en un diccionario de la forma:
### { enzima_i : mejor_copia_en_el_organismo_k }
### donde i recorre todas las enzimas en el metabolismo central, y k recorre todos los genomas.

In [8]:
df1 = pd.read_csv('/home/csar/Proyectos/Posdoc/Proyecto_pos/dev_package/data/data_bases/blastp/central_to_genomes/central_to_genomes.blast', sep = "\t", names = CTS.BLAST_COLS )

In [40]:
central_enzimes = df1['query'].unique() ##Lista con las enzimas del metabolismo central
genomes = hlp.dict_ids_to_names_genomes(CTS.EXMPL_DB["rast_ids"]) ## Lista con los genomas de la base de genomas

In [71]:
df1.iloc[0]

query            3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...
subject          gi|6666666.146852.1090|6666666.146852|NC_00293...
pc_identity                                                 79.623
aln_length                                                     530
mismatches                                                     108
gaps_opened                                                      0
query_start                                                      1
query_end                                                      530
subject_start                                                    1
subject_end                                                    530
e_value                                                        0.0
bitscore                                                     847.0
Name: 0, dtype: object

In [79]:
unique_in_first_blast = dict()
for enzime in central_enzimes:
    unique_in_first_blast[enzime] = []
    


aux_list = list()
for enzime in central_enzimes:
    for genome_id in genomes.keys():
        for index in range(len(df1)):
            if (genome_id in df1["subject"][index]) & (aux_list == []):
                aux_list=list(df1["subject"][index])
            elif (genome_id in df1["subject"][index]) & (df1["bitscore"][index] > aux_list[0][11]):
                aux_list = list(df1["subject"][index])
            else:
                continue
        unique_in_first_blast[enzime].append(aux_list)
    

IndexError: list index out of range